## Example: $d=2$ Rayleigh-Benard convection in a semicircle

$$\Omega = \{(x, y)~:~0 < x^2 + y^2 < R_{\text{outer}}^2~,~y>0\}$$

$$\partial\Omega_{\text{arc}} = \{(x, y)~:~ x^2 + y^2 = R^2 \}$$

$$\partial\Omega_{\text{lower}} = \{(x, y)~:~ y=0 \}$$

$$c_0(r)=\mathcal{N}(r)~~,~~r=\sqrt{x^2 + y^2}$$

$$c\vert_{\partial\Omega_{\text{lower}}}=1$$

$$c\vert_{\partial\Omega_{\text{arc}}}=0$$

$$\textbf{e}_g=-\textbf{e}_y$$

$$
\begin{align*}
\phi &= 1 \\
\mathsf{D} &= \mathsf{I} \\ 
\mathsf{K} &= \mathsf{I}\\ 
\mu &= 1 \\
\rho(c) &= -c
\end{align*}
$$

In [ ]:
import numpy as np
from ufl import SpatialCoordinate, sqrt
from lucifex.fdm import FiniteDifference, AB2, CN
from lucifex.mesh import arc_mesh, mesh_boundary
from lucifex.solver import BoundaryConditions, OptionsPETSc
from lucifex.sim import Simulation, integrate, configure_simulation
from lucifex.utils import CellType, SpatialPerturbation, triangulation
from lucifex.viz import plot_colormap
from lucifex.io import write

from py.porous_convection import create_simulation


@configure_simulation(
    store_step=1,
    write_step=None,
)
def rayleigh_benard_semicircle(
    radius: float,
    Nradial: int = 100,
    cell: str = CellType.TRIANGLE,
    Ra: float = 5e2,
    c_eps: float = 1e-6,
    c_freq: int = 8,
    dt_max: float = 0.5,
    cfl_h: str | float = "hmin",
    cfl_courant: float = 0.75,
    D_adv: FiniteDifference | tuple[FiniteDifference, FiniteDifference] = (AB2, CN),
    D_diff: FiniteDifference = CN,
    psi_petsc: OptionsPETSc | None = None,
    c_petsc: OptionsPETSc | None = None,
    secondary: bool = False,
):
    r2 = lambda x: x[0]**2 + x[1]**2
    r = lambda x, sqrt=np.sqrt: sqrt(r2(x))
    dr = radius / Nradial
    Omega = arc_mesh
    dOmega = mesh_boundary(
        Omega, 
        {
            "lower": lambda x: x[1],
            "outer": lambda x: r2(x) - radius**2,
        },
    )
    c_bcs = BoundaryConditions(
        ("dirichlet", dOmega['inner'], 1.0),
        ("dirichlet", dOmega['outer'], 0.0),  
    )
    radial_noise = lambda x: c_eps * np.sin(c_freq * np.pi * (r(x, np.sqrt) - Rinner) / (Router - Rinner))
    c_ics = SpatialPerturbation(
        lambda x: np.log(Router / r(x, np.sqrt)) / np.log(Router / Rinner),
        radial_noise,
        Omega.geometry.x,
        c_eps,
        ) 
    simulation = create_simulation(
        Omega=Omega, 
        dOmega=dOmega, 
        Ra=Ra, 
        c_ics=c_ics, 
        c_bcs=c_bcs, 
        density=density, 
        dt_max=dt_max, 
        cfl_h=cfl_h, 
        cfl_courant=cfl_courant,
        D_adv=D_adv, 
        D_diff=D_diff, 
        psi_petsc=psi_petsc, 
        c_petsc=c_petsc, 
        secondary=secondary,
    )
    return simulation